# NYC Motor Vehicle Crashes Dataset Cleaning

In [64]:
import pandas as pd
import numpy as np

# Load the crash data
df = pd.read_csv("Motor_Vehicle_Collisions_-_Crashes.csv", low_memory=False)
df.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,09/11/2021,9:35,BROOKLYN,11208,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,12/14/2021,8:13,BROOKLYN,11233,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN


In [65]:
# See number of null values in each column
df.isna().sum() 

CRASH DATE                             0
CRASH TIME                             0
BOROUGH                           641181
ZIP CODE                          641429
LATITUDE                          232765
LONGITUDE                         232765
LOCATION                          232765
ON STREET NAME                    436626
CROSS STREET NAME                 777123
OFF STREET NAME                  1716767
NUMBER OF PERSONS INJURED             18
NUMBER OF PERSONS KILLED              31
NUMBER OF PEDESTRIANS INJURED          0
NUMBER OF PEDESTRIANS KILLED           0
NUMBER OF CYCLIST INJURED              0
NUMBER OF CYCLIST KILLED               0
NUMBER OF MOTORIST INJURED             0
NUMBER OF MOTORIST KILLED              0
CONTRIBUTING FACTOR VEHICLE 1       6688
CONTRIBUTING FACTOR VEHICLE 2     318213
CONTRIBUTING FACTOR VEHICLE 3    1914038
CONTRIBUTING FACTOR VEHICLE 4    2027881
CONTRIBUTING FACTOR VEHICLE 5    2052056
COLLISION_ID                           0
VEHICLE TYPE COD

In [66]:
# Create a mapping from coordinates to zip codes for rows that have a zip code
coordinates_to_zip = df.dropna(subset=['ZIP CODE']).set_index('LOCATION')['ZIP CODE'].to_dict()

# Fill missing zip codes using the mapping
df['ZIP CODE'] = df.apply(lambda row: coordinates_to_zip.get(row['LOCATION'], row['ZIP CODE']), axis=1)

In [67]:
# Repeat same process for Borough
coordinates_to_borough = df.dropna(subset=['BOROUGH']).set_index('LOCATION')['BOROUGH'].to_dict()
df['BOROUGH'] = df.apply(lambda row: coordinates_to_borough.get(row['LOCATION'], row['BOROUGH']), axis=1)

In [68]:
# See updated number of null values for borough and zip code
df.isna().sum() 

CRASH DATE                             0
CRASH TIME                             0
BOROUGH                           251394
ZIP CODE                          251620
LATITUDE                          232765
LONGITUDE                         232765
LOCATION                          232765
ON STREET NAME                    436626
CROSS STREET NAME                 777123
OFF STREET NAME                  1716767
NUMBER OF PERSONS INJURED             18
NUMBER OF PERSONS KILLED              31
NUMBER OF PEDESTRIANS INJURED          0
NUMBER OF PEDESTRIANS KILLED           0
NUMBER OF CYCLIST INJURED              0
NUMBER OF CYCLIST KILLED               0
NUMBER OF MOTORIST INJURED             0
NUMBER OF MOTORIST KILLED              0
CONTRIBUTING FACTOR VEHICLE 1       6688
CONTRIBUTING FACTOR VEHICLE 2     318213
CONTRIBUTING FACTOR VEHICLE 3    1914038
CONTRIBUTING FACTOR VEHICLE 4    2027881
CONTRIBUTING FACTOR VEHICLE 5    2052056
COLLISION_ID                           0
VEHICLE TYPE COD

In [69]:
# Remove rows that are still null for borough and zip code (less than 15% of the data is null now)
df_cleaned = df.dropna(subset=['ZIP CODE'])
df_cleaned = df_cleaned.dropna(subset=['BOROUGH'])

# Remove rows that are null for vehicle type 1 and borough (still less than 15% of the data is being removed)
df_cleaned = df_cleaned.dropna(subset=['VEHICLE TYPE CODE 1'])
df_cleaned = df_cleaned.dropna(subset=['CONTRIBUTING FACTOR VEHICLE 1'])

In [70]:
# See updated number of null values for borough, zip code, vehicle code 1, contributing factor 1 again
df_cleaned.isna().sum() 

CRASH DATE                             0
CRASH TIME                             0
BOROUGH                                0
ZIP CODE                               0
LATITUDE                          231236
LONGITUDE                         231236
LOCATION                          231236
ON STREET NAME                    367300
CROSS STREET NAME                 572213
OFF STREET NAME                  1462597
NUMBER OF PERSONS INJURED             14
NUMBER OF PERSONS KILLED              23
NUMBER OF PEDESTRIANS INJURED          0
NUMBER OF PEDESTRIANS KILLED           0
NUMBER OF CYCLIST INJURED              0
NUMBER OF CYCLIST KILLED               0
NUMBER OF MOTORIST INJURED             0
NUMBER OF MOTORIST KILLED              0
CONTRIBUTING FACTOR VEHICLE 1          0
CONTRIBUTING FACTOR VEHICLE 2     273747
CONTRIBUTING FACTOR VEHICLE 3    1677802
CONTRIBUTING FACTOR VEHICLE 4    1767492
CONTRIBUTING FACTOR VEHICLE 5    1787009
COLLISION_ID                           0
VEHICLE TYPE COD

In [71]:
# Filling null values in "NUMBER OF PERSONS INJURED" with the sum of individual injury columns
df_cleaned['NUMBER OF PERSONS INJURED'] = df_cleaned['NUMBER OF PERSONS INJURED'].fillna(
    df_cleaned['NUMBER OF PEDESTRIANS INJURED'] + 
    df_cleaned['NUMBER OF CYCLIST INJURED'] + 
    df_cleaned['NUMBER OF MOTORIST INJURED']
)

# Filling null values in "NUMBER OF PERSONS KILLED" with the sum of individual killed columns
df_cleaned['NUMBER OF PERSONS KILLED'] = df_cleaned['NUMBER OF PERSONS KILLED'].fillna(
    df_cleaned['NUMBER OF PEDESTRIANS KILLED'] + 
    df_cleaned['NUMBER OF CYCLIST KILLED'] + 
    df_cleaned['NUMBER OF MOTORIST KILLED']
)

In [72]:
# See updated number of null values for "NUMBER OF PERSONS INJURED" and "NUMBER OF PERSONS KILLED"
df_cleaned.isna().sum() 

CRASH DATE                             0
CRASH TIME                             0
BOROUGH                                0
ZIP CODE                               0
LATITUDE                          231236
LONGITUDE                         231236
LOCATION                          231236
ON STREET NAME                    367300
CROSS STREET NAME                 572213
OFF STREET NAME                  1462597
NUMBER OF PERSONS INJURED              0
NUMBER OF PERSONS KILLED               0
NUMBER OF PEDESTRIANS INJURED          0
NUMBER OF PEDESTRIANS KILLED           0
NUMBER OF CYCLIST INJURED              0
NUMBER OF CYCLIST KILLED               0
NUMBER OF MOTORIST INJURED             0
NUMBER OF MOTORIST KILLED              0
CONTRIBUTING FACTOR VEHICLE 1          0
CONTRIBUTING FACTOR VEHICLE 2     273747
CONTRIBUTING FACTOR VEHICLE 3    1677802
CONTRIBUTING FACTOR VEHICLE 4    1767492
CONTRIBUTING FACTOR VEHICLE 5    1787009
COLLISION_ID                           0
VEHICLE TYPE COD

In [73]:
# Lowercase all vehicle types to make data cleaning easier
df_cleaned['VEHICLE TYPE CODE 1'] = df_cleaned['VEHICLE TYPE CODE 1'].str.lower()
df_cleaned['VEHICLE TYPE CODE 2'] = df_cleaned['VEHICLE TYPE CODE 2'].str.lower()

In [74]:
# Get all unique values in 'VEHICLE TYPE CODE 1' column
print(df_cleaned['VEHICLE TYPE CODE 1'].unique().tolist())

['sedan', 'dump', 'station wagon/sport utility vehicle', 'tanker', 'bus', 'taxi', 'van', 'motorscooter', 'bike', 'e-bike', 'ambulance', 'box truck', 'garbage or refuse', 'motorcycle', 'e-scooter', 'utility', 'convertible', 'flat bed', 'tow truck / wrecker', 'pick-up truck', '�mbu', 'moped', 'tractor truck diesel', 'trailer', 'school bus', 'lift boom', 'scooter', 'cater', 'pkup', 'carry all', 'fdny ambul', '3-door', 'beverage truck', 'semi trail', 'motorbike', 'unknown', 'lunch wagon', 'flat rack', 'unk', 'uspcs', 'garbage tr', 'minicycle', 'fire truck', 'armored truck', 'usps truck', 'com', 'chassis cab', 'pk', 'sw/van', 'concrete mixer', 'open body', 'excavator', 'fork lift', 'street swe', 'commercial', 'psd', '4 dr sedan', 'mta', 'truck', 'tow truck', 'e bike uni', 'usps vehic', 'fire', 'limo', 'tractor truck gasoline', 'mack', 'refrigerated van', 'stake or rack', 'work van', 'motor scoo', 'pickup tru', 'department', 'nypd van', 'pick up', 'ambulace', '2 dr sedan', 'fdny firet', 'pal

In [75]:
# Define a function that will place vehicle names into 10 different buckets
def categorize_vehicle_1(vehicle_type):
    if vehicle_type in ['ems/van','marked van','smart car', 'wh ford co','chevrolet','ford econo','yellow cab','van t','van/t','3 whe','cab', 'mobil','passenger vehicle', 'sport utility / station wagon', 'livery vehicle', 'petit cab', 'passanger', '4dsd', '4sedn', 'car/suv', 'toyot', 'mini vahn', 'subar', 'electric c', 'pass-', 'mta van', 'chevy', 'pas v', 'conve', 'chevr', 'chev', 'van (trans', 'ford trans', 'sm yw', 'van e','range', 'lmb', 'van (', 'yello', 'subur', 'e - b', 'navig', 'passe', 'sc', 'santa', 'te', 'nissa', 'van a', 'limou', 'coupe', 'van/b', 'highl', 'e-350', 'electric b', 'van w', 'passa', 'liver', 'rubbe', 'delie', 'miniv','sedan', 'station wagon/sport utility vehicle','taxi','van','convertible','carry all','3-door','sw/van','4 dr sedan','limo','2 dr sedan','mini van','ford van','sprinter v','passenger','passenger','pas','yellow cab','mercedes','toyota','12 passeng','sonata','jeep','van wh','cherv','4dsd','livery vehicle','limousine','rz','limou','sport utility / station wagon','coupe','4ds','yellow tax','station wagon/sport utility vehicle','convertible','minivan','car/suv','mini vahn','yellow tax','taxi', 'van','sonata','passanger','pass','suv','4 dr sedan','sedona','mini van','4sedn','passenger vehicle','2 dr sedan','pas v','pas (4dr s','sedan']:
        return 'Passenger Vehicle'
    elif vehicle_type in ['utility wh','fedex','usps 88716','postal van','ttailer','usps/govt','courier','moving van','delivery v','dot equipm', '18 wheeler','postal bus','utility.','delviery','usps #7530','wagon','pay loader','utili','dept van #','delvi','deliv','sanit','food', 'comme','santi','polic','parce', 'comm','fedx', 'small com veh(4 tires) ', 'large com veh(6 or more tires)', 'commerial', 'amazon tru', 'commer veh', 'fed ex tru', 'fleet', 'usps #6530', 'mail carri', 'commericia', 'pepsi deli', 'ups m', 'box car', 'ford couri', 'com v', 'scomm', 'dhl t', 'deiv', 'ecom', 'box h', 'scom', 'lcom', 'refr','comm.', 'couri', 'commu', 'u-hau', 'comer', 'ups t', 'usps2', 'frht', 'uspos', 'ems a', 'van c', 'fed e', 'e com', 'posto', 'heavy', 'cart', 'commm', 'c0mme', 'u.s. posta', 'renta', 'tram', 'mta u', 'del', 'usps # 487','utility', 'beverage truck','lunch wagon', 'uspcs','usps truck','com', 'chassis cab','open body','commercial', 'usps vehic', 'refrigerated van','stake or rack','work van','postal ser','uhal','cargo van','cement tru','delivery t','enclosed body - nonremovable enclosure','commercial','delv','limousine','tractor','trac','tractor tr','oml/omnibu','tow','bobcat','con ed tru','large com veh(6 or more tires)','verzion va','u-haul','u-haul min','commerial','boxtruck','food truck','postal veh','mailtruck','mail truck','ups t','freight tr','penske box','freightlin','freig','frieg','usps','usps 88716','usps truc,','postal van','usps # 487','usps2','u.s. posta','fedx','fedex box', 'fedex truc','delivery t','com delive','small com veh(4 tires) ','commericia','fedex','usps mail','usps van','usps  van', 'fed ex tru','delivery','usps truck','us postal','ups','box truck','box','mail truck','commerical','mail carri','usps #7530','amazon tru','usps','usps #6530','ups truck','postal tru','uspos','usps posta','usps small','postal ser','us mail tr','mail','post offic','usps vehic']:
        return 'Commercial Vehicle'
    elif vehicle_type in ['hertz ram','quality tr','ram promas','trk','mta truck','usps truc,','u haul tru','truck trai', 'ladder tru','truck van','dilevery t','food truck','flatbed','flat bed t', 'flatbed fr', 'flat  bed','f150xl pic','t880','tr-trailer','i-haul','trail', 'tow t','towe truck','uhual', 'picku','fuel','trc', 'garba','flat/','towtr','pick-','uhaul','f650','waste truc', 'ford f150', 'ford f-150', 'pickup tow', 'truck w/tr', 'dot truck', 'fd truck', 'dot vehicl', 'flatbed to', 'tow-truck', 'u-hal', 'dot', 'box van', 'heavy truc', 'freight va', 'e tow', 'gmc ecom', 'stake', 'flat-', 'p/u', 'towma','ice c', 'barri', 'movin', 'const', 'bucke', 'tktr', 'mac t', 'mac t', 'red t', 'dumpt', 'flatb', 'rd/s', 'broom', 'fd tr', 'sweep', 'cemen', 'icecr', 'tow-t', 'g sem', 'oil t', 'ladde', 'refri', 'sterl', 'tl tr', 'cabin', 'nt tr','tanker','box truck','flat bed', 'tow truck / wrecker','pick-up truck','tractor truck diesel','pkup', 'garbage tr','truck','tow truck','tractor truck gasoline','mack','pickup tru','pick up','mailtruck','pick up tr','freight','hopper','pickup','dump truck','semi','truck comm','mack truck','flatbed pi','pick-up tr','grumman ll','work truck','tk','sprinter w','g com','promaster','vanette','econoline','van truck','f550','f550 esu r','tlr','tlc','moving tru','dodge ram','ram','pick wh','pick-up','flat bed t','flat','garbage tr','fd truck','flat  bed','pickup with mounted camper','ford f150','ford picku','flatbed fr','flatb','beverage truck','pick-up truck','flat bed', 'tow truck / wrecker','heavy truc','garbage or refuse','flatbed pi','flatbed','ice cream','pick up tr','pick up','dumptruck','dumpster t','pickup tru','garbage','pick-up tr','utility tr','dump truck','pickup','dump']:
        return 'Truck'
    elif vehicle_type in ['subn - amb','g amb','nyc ems','leased amb','almbulance','abulance','fdny #226','firet truc','firtruck','gen  ambul','nyc fd','white ambu','emergency','esu rescue','ambul','amulance','firet','nyfd', 'ford ambul', 'ambalance', 'nyc fdny #', 'ambulanve', 'fd ambulan', 'police veh', 'fire truvk', 'fdny mecha', 'fdny- fire', 'fdny utili', 'fdny vehic', 'ems t', 'e.m.s', 'ems b', 'x amb', 'firer','rescu', 'e amb', 'emrgn', 'amabu', 'abula','ambulance', 'fdny ambul', 'fire truck', 'fire','nypd van','ambulace','fdny firet','ambulane', 'amb','ambu','ems bus','firetruck','fdny','nys ambula','ems','pumper','fdny fire','ambulence','fdny truck','fire','fire engine','fdny truck','city vehic','city of ne','fdny ambul','nypd','nypd tow t','ambulette','police veh','police rep','unmarked v','ems t','firet truc','e.m.s','polic','nypd van','fdny','nyc fd','nycha van','fdny ems v','ems a','firet','nyc ems','fdny fire','ambalance','ford ambul','amubulance','ambulane', 'amb','nyc fdny #','almbulance','fdny firet','firetruck','ambulance','ems ambula','nyc fire t','fdny lader','fdny ladde','fdny #226','white ambu','fdny ems','fdny utili','fdny vehic','gen  ambul','nyc ambula','fd ambulan','ems fdny b','fdny engine','ems truck','emergency','emt ambula','ems','ems bus','amulance','ambukance','firtruck','fire truvk','ambulanve','ambulence','ambulace','ambu','abulance','nys ambula','fire truck']:
        return 'Emergency Vehicle'
    elif vehicle_type in ['schoo','yellow bus', 'omnibus', 'bus (omnib', 'small bus', 'truck/bus', 'coach', 'mta transi', 'metro tran', 'mta v', 'marked rmp','trans', 'omnib', '12 pa', 'chart', 'nyc bus', 'omni', 'hino', 'livery bus','bus', 'school bus', 'shuttle bu','12 passage','mta bus','shuttle bu','trolley ri','livery bus','nyc bus','small bus','truck/bus','small bus','bus (omnib','schoolbus','mini schoo','school  bu','yellow bus','school bus','school bu','mini bus', 'mta bus','bus']:
        return 'Bus'
    elif vehicle_type in ['spark150 s','ebike','bicycle','e bike','elec. unic','e-bik','push scoot','e-sco','e bik','scoot','vespa','mopped', 'pedal bike', 'revel mope', 'ele motorc', 'gas moped', 'razor scoo', 'ele. unicy', 'e- bi', 'e motorcyc', 'electric m', '2 whe', 'boom moped', '50cc mini', 'flywing mo', '2 wheel sc','e sco', 'motor', 'moter', 'bicyc', 'mc', 'e-mot', 'mcy', 'mo pa', 'dirtb', 'mtr s', 'winne', 'mcy b','motorscooter','bike','e-bike', 'motorcycle', 'e-scooter','moped', 'scooter', 'motorbike', 'minicycle', 'motor scoo','e scooter','50cc scoot','kick scoot','electric s','dirt bike','minibike','escooter s','motorized','gas scoote','crf carria','revel scoo','moped','ebike','gas mo ped','e bike','bicycle','scoot','razor150 m','motorcycle','yamah','red moped','e motorcyc','bicyc','mopd','boom moped','gas mo ped','yamaha','e bike w p', 'e-scooter','minicycle','motorbike','scooter ga','ele motorc','gas moped','electric b','e bike uni','gas scoote','scooter','motor scoo','mopped','dirt bike','razor scoo', 'e-bik','pedal bike','motorscoot','electric s','minibike','escooter s','50cc scoot','kick scoot','e scooter','bike', 'e-bike','motorscooter']:
        return 'Two-Wheeled Vehicle'
    elif vehicle_type in ['skid loade','bulldozer','forlift','catapillar','drill rig','john deere','back hoe','mechanical','cat','escavator','tract','fork','excav','yale fork', 'bkhoe', 'hi-lo','skid-', 'kubot', 'bulld', 'trlr', 'track exca', 'concr', 'dumps', 'cat p', 'boom', 'rmp', 'cherr', 'bobca','dump','garbage or refuse', 'lift boom','cater','concrete mixer', 'excavator','fork lift','forklift','asphalt ro','power shov','cat forkli','fork','boom lift','stak','bucket tru','power ladd','trailler/fl','backhoe re','well driller','backhoe lo','fork lift','crane boom','forkl','constructi','cat forkli','forlift','crane','concrete mixer','bulldozer','forklift','cement tru'] :
        return 'Construction Vehicle'
    elif vehicle_type in ['trailee','camper tra','rv/van','motorhome', 'rv/tr', 'emi- trail', 'revel','sea', 'camp', 'r/v c', 'r/v','rv','motorized home','house on w','hwh','r/v','rv','r/v c','rv/van','van camper','motorhome','camper van','motor home','jet-ski']:
        return 'Recreational Vehicle'
    elif vehicle_type in ['special co','sanitaion','sanitaton','lawnmower','e-skateboa','go kart','horse','hrse','golf','tanke','golfcart', 'military v', 'utv bobcat', 'sweeper', 'govt p/u t', 'gokar', 'govt.', 'horse draw', 'farm', 'military', 'hosre', 'ladder', 'atv p', 'servi', 'trac.', 'swepe', 'wag', 'traffic to','nys a', 'dirt', 'segwa', 'nycha', 'snow', 'grain', 'gover', 'nyc d', 'track', 'fd fi', 'spc', 'light', 'tower', 'utv', 'nyc', 'loade', 'cat 9', 'fork-', 'trlr plt,', 'spec', 'seagr', 'cargo', 'nyc a', 'dot r', 'veriz', 'farm vehicle', 'carri', 'dp', 'gov v', 'nyc f', 'nycta','trailer','semi trail', 'flat rack', 'armored truck', 'street swe','e bike uni','golf cart','util','bulk agriculture','golf car','livestock rack','road sweep','van/truck','street cle','lift','skywatch','carriage','pedicab','snow plow','horse trai','utility ve','utility tr','golfcart','golf cart','armored truck','golf car','locomotive','skateboard','sanitation','horse carr','food cart','government','govt','principle']:
        return 'Specialized or Miscellaneous Vehicle'
    else:
        return 'Unknown/Other'

In [76]:
# Apply the function to each value in 'vehicle_type 1' column
df_cleaned['VEHICLE TYPE CODE 1'] = df_cleaned['VEHICLE TYPE CODE 1'].apply(categorize_vehicle_1)

In [77]:
# Make sure all vehicle types have been categorized in 'VEHICLE TYPE CODE 1'
print(df_cleaned['VEHICLE TYPE CODE 1'].unique().tolist())

['Passenger Vehicle', 'Truck', 'Bus', 'Two-Wheeled Vehicle', 'Emergency Vehicle', 'Commercial Vehicle', 'Unknown/Other', 'Specialized or Miscellaneous Vehicle', 'Construction Vehicle', 'Recreational Vehicle']


In [78]:
pd.set_option('display.max_rows', None)

# Print the value counts for the cleaned 'VEHICLE TYPE CODE 1' column
value_counts_1 = df_cleaned['VEHICLE TYPE CODE 1'].value_counts()
print(value_counts_1)

Passenger Vehicle                       1566137
Truck                                     57627
Commercial Vehicle                        48898
Unknown/Other                             44175
Two-Wheeled Vehicle                       33302
Bus                                       33114
Emergency Vehicle                          8966
Specialized or Miscellaneous Vehicle       1219
Construction Vehicle                        895
Recreational Vehicle                         97
Name: VEHICLE TYPE CODE 1, dtype: int64


In [79]:
# Now see all unique values in 'VEHICLE TYPE CODE 2'
print(df_cleaned['VEHICLE TYPE CODE 2'].unique().tolist())

['sedan', nan, 'pick-up truck', 'station wagon/sport utility vehicle', 'bike', 'e-scooter', 'e-bike', 'ambulance', 'moped', 'bus', 'motorcycle', 'taxi', 'box van', '4 dr sedan', 'tanker', 'box truck', 'dl', 'refrigerated van', 'tow truck / wrecker', 'tractor truck diesel', 'garbage or refuse', 'chassis cab', 'flat rack', 'van', 'pk', 'dump', 'tractor truck gasoline', 'motorbike', 'concrete mixer', 'multi-wheeled vehicle', 'fire truck', 'hopper', 'fdny truck', 'van/truck', 'pedicab', 'flat bed', 'motorscooter', 'fdny engin', 'carry all', 'convertible', 'dirt bike', 'firetruck', 'util truck', 'armored truck', 'box', 'nyc fire t', 'van camper', 'lift boom', 'scooter', 'wheelchair', 'tow truck', 'stake or rack', 'food cart', '999', 'unk', 'beverage truck', 'power shov', 'work van', 'nice bus', 'trailer', '3-door', 'open body', 'mta bus', 'trk', 'com', 'forklift', 'garbage tr', 'bmw', 'snow plow', 'school bus', 'escootersi', 'bulk agriculture', 'mower', 'ez go', 'wh', 'usps', 'minibike', 'u

In [80]:
# Now we will define a similar function that will group vehicle names into buckets in 'VEHICLE TYPE CODE 2'.
# But we don't want to categorize anything as Unknown/Other yet
def categorize_vehicle_2(vehicle_type):
    if vehicle_type in ['ems/van','marked van','smart car', 'wh ford co','chevrolet','ford econo','yellow cab','van t','van/t','3 whe','cab', 'mobil','passenger vehicle', 'sport utility / station wagon', 'livery vehicle', 'petit cab', 'passanger', '4dsd', '4sedn', 'car/suv', 'toyot', 'mini vahn', 'subar', 'electric c', 'pass-', 'mta van', 'chevy', 'pas v', 'conve', 'chevr', 'chev', 'van (trans', 'ford trans', 'sm yw', 'van e','range', 'lmb', 'van (', 'yello', 'subur', 'e - b', 'navig', 'passe', 'sc', 'santa', 'te', 'nissa', 'van a', 'limou', 'coupe', 'van/b', 'highl', 'e-350', 'electric b', 'van w', 'passa', 'liver', 'rubbe', 'delie', 'miniv','sedan', 'station wagon/sport utility vehicle','taxi','van','convertible','carry all','3-door','sw/van','4 dr sedan','limo','2 dr sedan','mini van','ford van','sprinter v','passenger','passenger','pas','yellow cab','mercedes','toyota','12 passeng','sonata','jeep','van wh','cherv','4dsd','livery vehicle','limousine','rz','limou','sport utility / station wagon','coupe','4ds','yellow tax','station wagon/sport utility vehicle','convertible','minivan','car/suv','mini vahn','yellow tax','taxi', 'van','sonata','passanger','pass','suv','4 dr sedan','sedona','mini van','4sedn','passenger vehicle','2 dr sedan','pas v','pas (4dr s','sedan']:
        return 'Passenger Vehicle'
    elif vehicle_type in ['utility wh','fedex','usps 88716','postal van','ttailer','usps/govt','courier','moving van','delivery v','dot equipm', '18 wheeler','postal bus','utility.','delviery','usps #7530','wagon','pay loader','utili','dept van #','delvi','deliv','sanit','food', 'comme','santi','polic','parce', 'comm','fedx', 'small com veh(4 tires) ', 'large com veh(6 or more tires)', 'commerial', 'amazon tru', 'commer veh', 'fed ex tru', 'fleet', 'usps #6530', 'mail carri', 'commericia', 'pepsi deli', 'ups m', 'box car', 'ford couri', 'com v', 'scomm', 'dhl t', 'deiv', 'ecom', 'box h', 'scom', 'lcom', 'refr','comm.', 'couri', 'commu', 'u-hau', 'comer', 'ups t', 'usps2', 'frht', 'uspos', 'ems a', 'van c', 'fed e', 'e com', 'posto', 'heavy', 'cart', 'commm', 'c0mme', 'u.s. posta', 'renta', 'tram', 'mta u', 'del', 'usps # 487','utility', 'beverage truck','lunch wagon', 'uspcs','usps truck','com', 'chassis cab','open body','commercial', 'usps vehic', 'refrigerated van','stake or rack','work van','postal ser','uhal','cargo van','cement tru','delivery t','enclosed body - nonremovable enclosure','commercial','delv','limousine','tractor','trac','tractor tr','oml/omnibu','tow','bobcat','con ed tru','large com veh(6 or more tires)','verzion va','u-haul','u-haul min','commerial','boxtruck','food truck','postal veh','mailtruck','mail truck','ups t','freight tr','penske box','freightlin','freig','frieg','usps','usps 88716','usps truc,','postal van','usps # 487','usps2','u.s. posta','fedx','fedex box', 'fedex truc','delivery t','com delive','small com veh(4 tires) ','commericia','fedex','usps mail','usps van','usps  van', 'fed ex tru','delivery','usps truck','us postal','ups','box truck','box','mail truck','commerical','mail carri','usps #7530','amazon tru','usps','usps #6530','ups truck','postal tru','uspos','usps posta','usps small','postal ser','us mail tr','mail','post offic','usps vehic']:
        return 'Commercial Vehicle'
    elif vehicle_type in ['hertz ram','quality tr','ram promas','trk','mta truck','usps truc,','u haul tru','truck trai', 'ladder tru','truck van','dilevery t','food truck','flatbed','flat bed t', 'flatbed fr', 'flat  bed','f150xl pic','t880','tr-trailer','i-haul','trail', 'tow t','towe truck','uhual', 'picku','fuel','trc', 'garba','flat/','towtr','pick-','uhaul','f650','waste truc', 'ford f150', 'ford f-150', 'pickup tow', 'truck w/tr', 'dot truck', 'fd truck', 'dot vehicl', 'flatbed to', 'tow-truck', 'u-hal', 'dot', 'box van', 'heavy truc', 'freight va', 'e tow', 'gmc ecom', 'stake', 'flat-', 'p/u', 'towma','ice c', 'barri', 'movin', 'const', 'bucke', 'tktr', 'mac t', 'mac t', 'red t', 'dumpt', 'flatb', 'rd/s', 'broom', 'fd tr', 'sweep', 'cemen', 'icecr', 'tow-t', 'g sem', 'oil t', 'ladde', 'refri', 'sterl', 'tl tr', 'cabin', 'nt tr','tanker','box truck','flat bed', 'tow truck / wrecker','pick-up truck','tractor truck diesel','pkup', 'garbage tr','truck','tow truck','tractor truck gasoline','mack','pickup tru','pick up','mailtruck','pick up tr','freight','hopper','pickup','dump truck','semi','truck comm','mack truck','flatbed pi','pick-up tr','grumman ll','work truck','tk','sprinter w','g com','promaster','vanette','econoline','van truck','f550','f550 esu r','tlr','tlc','moving tru','dodge ram','ram','pick wh','pick-up','flat bed t','flat','garbage tr','fd truck','flat  bed','pickup with mounted camper','ford f150','ford picku','flatbed fr','flatb','beverage truck','pick-up truck','flat bed', 'tow truck / wrecker','heavy truc','garbage or refuse','flatbed pi','flatbed','ice cream','pick up tr','pick up','dumptruck','dumpster t','pickup tru','garbage','pick-up tr','utility tr','dump truck','pickup','dump']:
        return 'Truck'
    elif vehicle_type in ['subn - amb','g amb','nyc ems','leased amb','almbulance','abulance','fdny #226','firet truc','firtruck','gen  ambul','nyc fd','white ambu','emergency','esu rescue','ambul','amulance','firet','nyfd', 'ford ambul', 'ambalance', 'nyc fdny #', 'ambulanve', 'fd ambulan', 'police veh', 'fire truvk', 'fdny mecha', 'fdny- fire', 'fdny utili', 'fdny vehic', 'ems t', 'e.m.s', 'ems b', 'x amb', 'firer','rescu', 'e amb', 'emrgn', 'amabu', 'abula','ambulance', 'fdny ambul', 'fire truck', 'fire','nypd van','ambulace','fdny firet','ambulane', 'amb','ambu','ems bus','firetruck','fdny','nys ambula','ems','pumper','fdny fire','ambulence','fdny truck','fire','fire engine','fdny truck','city vehic','city of ne','fdny ambul','nypd','nypd tow t','ambulette','police veh','police rep','unmarked v','ems t','firet truc','e.m.s','polic','nypd van','fdny','nyc fd','nycha van','fdny ems v','ems a','firet','nyc ems','fdny fire','ambalance','ford ambul','amubulance','ambulane', 'amb','nyc fdny #','almbulance','fdny firet','firetruck','ambulance','ems ambula','nyc fire t','fdny lader','fdny ladde','fdny #226','white ambu','fdny ems','fdny utili','fdny vehic','gen  ambul','nyc ambula','fd ambulan','ems fdny b','fdny engine','ems truck','emergency','emt ambula','ems','ems bus','amulance','ambukance','firtruck','fire truvk','ambulanve','ambulence','ambulace','ambu','abulance','nys ambula','fire truck']:
        return 'Emergency Vehicle'
    elif vehicle_type in ['schoo','yellow bus', 'omnibus', 'bus (omnib', 'small bus', 'truck/bus', 'coach', 'mta transi', 'metro tran', 'mta v', 'marked rmp','trans', 'omnib', '12 pa', 'chart', 'nyc bus', 'omni', 'hino', 'livery bus','bus', 'school bus', 'shuttle bu','12 passage','mta bus','shuttle bu','trolley ri','livery bus','nyc bus','small bus','truck/bus','small bus','bus (omnib','schoolbus','mini schoo','school  bu','yellow bus','school bus','school bu','mini bus', 'mta bus','bus']:
        return 'Bus'
    elif vehicle_type in ['spark150 s','ebike','bicycle','e bike','elec. unic','e-bik','push scoot','e-sco','e bik','scoot','vespa','mopped', 'pedal bike', 'revel mope', 'ele motorc', 'gas moped', 'razor scoo', 'ele. unicy', 'e- bi', 'e motorcyc', 'electric m', '2 whe', 'boom moped', '50cc mini', 'flywing mo', '2 wheel sc','e sco', 'motor', 'moter', 'bicyc', 'mc', 'e-mot', 'mcy', 'mo pa', 'dirtb', 'mtr s', 'winne', 'mcy b','motorscooter','bike','e-bike', 'motorcycle', 'e-scooter','moped', 'scooter', 'motorbike', 'minicycle', 'motor scoo','e scooter','50cc scoot','kick scoot','electric s','dirt bike','minibike','escooter s','motorized','gas scoote','crf carria','revel scoo','moped','ebike','gas mo ped','e bike','bicycle','scoot','razor150 m','motorcycle','yamah','red moped','e motorcyc','bicyc','mopd','boom moped','gas mo ped','yamaha','e bike w p', 'e-scooter','minicycle','motorbike','scooter ga','ele motorc','gas moped','electric b','e bike uni','gas scoote','scooter','motor scoo','mopped','dirt bike','razor scoo', 'e-bik','pedal bike','motorscoot','electric s','minibike','escooter s','50cc scoot','kick scoot','e scooter','bike', 'e-bike','motorscooter']:
        return 'Two-Wheeled Vehicle'
    elif vehicle_type in ['skid loade','bulldozer','forlift','catapillar','drill rig','john deere','back hoe','mechanical','cat','escavator','tract','fork','excav','yale fork', 'bkhoe', 'hi-lo','skid-', 'kubot', 'bulld', 'trlr', 'track exca', 'concr', 'dumps', 'cat p', 'boom', 'rmp', 'cherr', 'bobca','dump','garbage or refuse', 'lift boom','cater','concrete mixer', 'excavator','fork lift','forklift','asphalt ro','power shov','cat forkli','fork','boom lift','stak','bucket tru','power ladd','trailler/fl','backhoe re','well driller','backhoe lo','fork lift','crane boom','forkl','constructi','cat forkli','forlift','crane','concrete mixer','bulldozer','forklift','cement tru'] :
        return 'Construction Vehicle'
    elif vehicle_type in ['trailee','camper tra','rv/van','motorhome', 'rv/tr', 'emi- trail', 'revel','sea', 'camp', 'r/v c', 'r/v','rv','motorized home','house on w','hwh','r/v','rv','r/v c','rv/van','van camper','motorhome','camper van','motor home','jet-ski']:
        return 'Recreational Vehicle'
    elif vehicle_type in ['special co','sanitaion','sanitaton','lawnmower','e-skateboa','go kart','horse','hrse','golf','tanke','golfcart', 'military v', 'utv bobcat', 'sweeper', 'govt p/u t', 'gokar', 'govt.', 'horse draw', 'farm', 'military', 'hosre', 'ladder', 'atv p', 'servi', 'trac.', 'swepe', 'wag', 'traffic to','nys a', 'dirt', 'segwa', 'nycha', 'snow', 'grain', 'gover', 'nyc d', 'track', 'fd fi', 'spc', 'light', 'tower', 'utv', 'nyc', 'loade', 'cat 9', 'fork-', 'trlr plt,', 'spec', 'seagr', 'cargo', 'nyc a', 'dot r', 'veriz', 'farm vehicle', 'carri', 'dp', 'gov v', 'nyc f', 'nycta','trailer','semi trail', 'flat rack', 'armored truck', 'street swe','e bike uni','golf cart','util','bulk agriculture','golf car','livestock rack','road sweep','van/truck','street cle','lift','skywatch','carriage','pedicab','snow plow','horse trai','utility ve','utility tr','golfcart','golf cart','armored truck','golf car','locomotive','skateboard','sanitation','horse carr','food cart','government','govt','principle']:
        return 'Specialized or Miscellaneous Vehicle'
    else:
        return vehicle_type

In [81]:
# Apply this second function to each row in 'vehicle_type 2' column
df_cleaned['VEHICLE TYPE CODE 2'] = df_cleaned['VEHICLE TYPE CODE 2'].apply(categorize_vehicle_2)

In [82]:
# Get remaining un-cleaned unique values in 'VEHICLE TYPE CODE 2' column
print(df_cleaned['VEHICLE TYPE CODE 2'].unique().tolist())

['Passenger Vehicle', nan, 'Truck', 'Two-Wheeled Vehicle', 'Emergency Vehicle', 'Bus', 'Commercial Vehicle', 'dl', 'Specialized or Miscellaneous Vehicle', 'pk', 'Construction Vehicle', 'multi-wheeled vehicle', 'fdny engin', 'util truck', 'Recreational Vehicle', 'wheelchair', '999', 'unk', 'nice bus', 'bmw', 'escootersi', 'mower', 'ez go', 'wh', 'unknown', 'refg', 'dirtbike', 'self insur', 'gov vehicl', 'ambulcance', 'fdny rig', 'mopad', 'courier va', 'mta nyc bu', 'tank', 'subn', 'mtabus', 'sprinter', 'unkn', 'ltrl', 'skatboard', 'toolcat', 'crane truc', 'unknow', 'other', 'uhaul truc', 'suburban', 'nycfiredep', 'cat payloa', "revel mo'e", 'garage', 'blue chevy', 'escavater', 'co', 'fdny ambu', 'm21', 'fire appar', 'special pu', 'bay crane', 'pallet', 'mini', 'gas bicycl', 'conedison', 'pedestrian', 'truck ladd', 'fire turck', 'moped clas', 'uhaul van', 'orange jlg', 'van box tr', 'limit use', 'food deliv', 'stake truc', 'boom lift/', 'pc', 'priv ambul', 'e skateboa', 'ec3', 'unk femal

In [83]:
# Define a function that will place remaining vehicle 2 names into 10 different buckets
def categorize_vehicle_3(vehicle_type):
    if pd.isna(vehicle_type): # Keep null values null
        return np.nan 
    if vehicle_type in ['Passenger Vehicle', 'school van', 'sprinter', 'subn', 'suburban', 'white van', 'sprin', 'sbn', 'livery omn']:
        return 'Passenger Vehicle'
    elif vehicle_type in ['Commercial Vehicle', 'ups van', 'fedex van', 'usps self', 'usps#', 'usps post', 'postoffice', 'usps box t', 'comm vehic', 'uspst', 'comm van', 'comm truck', 'posta', 'postal car', 'u.s p', 'us po', 'uspcs truc', 'post', 'fed ex', 'fed ex van', 'amazon van', 'courier va']:
        return 'Commercial Vehicle'
    elif vehicle_type in ['Truck', 'uhaul truc', 'uhaul van', '18 wheel', 'uhaulwtowd', 'u-haul tru', 'rental tru', 'refriger t', 'pick truck', 'flatbed tr', '18 weeler', 'u haul tra', 'pk', 'van box tr', 'box t', 'pick', 'u haul', 'refg', 'uhaul trai', 'dsny truck', 'towtruck', 'nyc truck', 'boxtr', 'truc']:
        return 'Truck'
    elif vehicle_type in ['Emergency Vehicle', 'fdny engin', 'emt truck', 'embulance', 'fire depar', 'nyc fire d', 'nyc fdny', 'fdny super', 'police van', 'priv ambul', 'fdny ambu', 'ambiance', 'ambulamce', 'ambluance', 'fdny picku', 'nycfiredep', 'ambulcance', 'abmu','fire turck', 'a bulance']:
        return 'Emergency Vehicle'
    elif vehicle_type in ['Bus', 'nice bus', 'charter bu', 'short bus', 'mtabus', 'mta bus  4', 'bus m', 'buss', 'blu bus', 'mta bus 40', 'fdny bus', 'mta nyc bu', 'mta b', 'city mta b', 'mta']:
        return 'Bus'
    elif vehicle_type in ['Two-Wheeled Vehicle', 'escootersi', 'gas bicycl', 'e-unicycle', 'citibike', 'unicycle', 'uni e-bike', 'moped bike', 'motor dirt', 'dart bike', 'mo ped', 'moped scoo', 'gas bike', 'scooter si', 'scooter/mo', '50 cc moto', 'mopet', 'mop pad', 'moto-scoot', 'mopoed', 'razor', 'moped clas', 'scooter no', 'dirtbike', 'escooter', 'scooter bi', 'e- scooter', 'scooter (g', 'mopad', 'moped/scoo', 'moped elec', 'moped gas', 'gas mo-ped', 'e-sooter', 'e/bik', 'electricun']:
        return 'Two-Wheeled Vehicle'
    elif vehicle_type in ['Construction Vehicle', 'contructio', 'forklift t', 'buldozer', 'gth-1056', 'pallet', 'back ho', 'backh', 'backhoe', 'back', 'crane truc', 'bay crane', 'bull dozer'] :
        return 'Construction Vehicle'
    elif vehicle_type in ['Recreational Vehicle', 'x trailer', 'semitraile', 'trailor', 'trl', 'autotrlr t','fltrl', 'trailer tr', 'semi-trail', 'semi-', 'trailer no']:
        return 'Recreational Vehicle'
    elif vehicle_type in ['Specialized or Miscellaneous Vehicle', 'gov vehicl', 'utility ca', 'hoverboard', 'snowmobile', 'boat', 'tracktor t', 'tank', 'armored tr', 'armor', 'wheelchair', 'util truck', 'golf kart', 'snow plowe', 'plow  truc', 'plow truck']:
        return 'Specialized or Miscellaneous Vehicle'
    else:
        return 'Unknown/Other'

In [84]:
# Apply this third function to each row in 'vehicle_type 2' column
df_cleaned['VEHICLE TYPE CODE 2'] = df_cleaned['VEHICLE TYPE CODE 2'].apply(categorize_vehicle_3)

In [85]:
# Make sure all vehicle types have been categorized in 'VEHICLE TYPE CODE 2'
print(df_cleaned['VEHICLE TYPE CODE 2'].unique().tolist())

['Passenger Vehicle', nan, 'Truck', 'Two-Wheeled Vehicle', 'Emergency Vehicle', 'Bus', 'Commercial Vehicle', 'Unknown/Other', 'Specialized or Miscellaneous Vehicle', 'Construction Vehicle', 'Recreational Vehicle']


In [86]:
# Print the value counts for the cleaned 'VEHICLE TYPE CODE 2' column
value_counts_2 = df_cleaned['VEHICLE TYPE CODE 2'].value_counts()
print(value_counts_2)

Passenger Vehicle                       1142125
Unknown/Other                            100589
Two-Wheeled Vehicle                       71236
Truck                                     55676
Commercial Vehicle                        51086
Bus                                       28414
Emergency Vehicle                          5324
Specialized or Miscellaneous Vehicle       1300
Construction Vehicle                       1148
Recreational Vehicle                        136
Name: VEHICLE TYPE CODE 2, dtype: int64


In [87]:
# See final number of null values in each column
df_cleaned.isna().sum() 

CRASH DATE                             0
CRASH TIME                             0
BOROUGH                                0
ZIP CODE                               0
LATITUDE                          231236
LONGITUDE                         231236
LOCATION                          231236
ON STREET NAME                    367300
CROSS STREET NAME                 572213
OFF STREET NAME                  1462597
NUMBER OF PERSONS INJURED              0
NUMBER OF PERSONS KILLED               0
NUMBER OF PEDESTRIANS INJURED          0
NUMBER OF PEDESTRIANS KILLED           0
NUMBER OF CYCLIST INJURED              0
NUMBER OF CYCLIST KILLED               0
NUMBER OF MOTORIST INJURED             0
NUMBER OF MOTORIST KILLED              0
CONTRIBUTING FACTOR VEHICLE 1          0
CONTRIBUTING FACTOR VEHICLE 2     273747
CONTRIBUTING FACTOR VEHICLE 3    1677802
CONTRIBUTING FACTOR VEHICLE 4    1767492
CONTRIBUTING FACTOR VEHICLE 5    1787009
COLLISION_ID                           0
VEHICLE TYPE COD

In [88]:
df_cleaned['LATITUDE'].fillna(-999.0000, inplace=True)
df_cleaned['LONGITUDE'].fillna(-999.0000, inplace=True)
df_cleaned['LOCATION'].fillna('NA', inplace=True)
df_cleaned['ON STREET NAME'].fillna('NA', inplace=True)
df_cleaned['OFF STREET NAME'].fillna('NA', inplace=True)
df_cleaned['CROSS STREET NAME'].fillna('NA', inplace=True)
df_cleaned['CONTRIBUTING FACTOR VEHICLE 2'].fillna('NA', inplace=True)
df_cleaned['CONTRIBUTING FACTOR VEHICLE 3'].fillna('NA', inplace=True)
df_cleaned['CONTRIBUTING FACTOR VEHICLE 4'].fillna('NA', inplace=True)
df_cleaned['CONTRIBUTING FACTOR VEHICLE 5'].fillna('NA', inplace=True)
df_cleaned['VEHICLE TYPE CODE 2'].fillna('Unknown/Other', inplace=True)
df_cleaned['VEHICLE TYPE CODE 3'].fillna('NA', inplace=True)
df_cleaned['VEHICLE TYPE CODE 4'].fillna('NA', inplace=True)
df_cleaned['VEHICLE TYPE CODE 5'].fillna('NA', inplace=True)

In [89]:
# See final number of null values in each column
df_cleaned.isna().sum() 

CRASH DATE                       0
CRASH TIME                       0
BOROUGH                          0
ZIP CODE                         0
LATITUDE                         0
LONGITUDE                        0
LOCATION                         0
ON STREET NAME                   0
CROSS STREET NAME                0
OFF STREET NAME                  0
NUMBER OF PERSONS INJURED        0
NUMBER OF PERSONS KILLED         0
NUMBER OF PEDESTRIANS INJURED    0
NUMBER OF PEDESTRIANS KILLED     0
NUMBER OF CYCLIST INJURED        0
NUMBER OF CYCLIST KILLED         0
NUMBER OF MOTORIST INJURED       0
NUMBER OF MOTORIST KILLED        0
CONTRIBUTING FACTOR VEHICLE 1    0
CONTRIBUTING FACTOR VEHICLE 2    0
CONTRIBUTING FACTOR VEHICLE 3    0
CONTRIBUTING FACTOR VEHICLE 4    0
CONTRIBUTING FACTOR VEHICLE 5    0
COLLISION_ID                     0
VEHICLE TYPE CODE 1              0
VEHICLE TYPE CODE 2              0
VEHICLE TYPE CODE 3              0
VEHICLE TYPE CODE 4              0
VEHICLE TYPE CODE 5 

In [90]:
# Preview the data
df_cleaned.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,BRONX,10463,-999.000000,-999.0000,NA,WHITESTONE EXPRESSWAY,20 AVENUE,NA,...,Unspecified,NA,NA,NA,4455765,Passenger Vehicle,Passenger Vehicle,NA,NA,NA
1,03/26/2022,11:45,BRONX,10463,-999.000000,-999.0000,NA,QUEENSBORO BRIDGE UPPER,NA,NA,...,NA,NA,NA,NA,4513547,Passenger Vehicle,Unknown/Other,NA,NA,NA
2,06/29/2022,6:55,BRONX,10463,-999.000000,-999.0000,NA,THROGS NECK BRIDGE,NA,NA,...,Unspecified,NA,NA,NA,4541903,Passenger Vehicle,Truck,NA,NA,NA
3,09/11/2021,9:35,BROOKLYN,11208,40.667202,-73.8665,"(40.667202, -73.8665)",NA,NA,1211 LORING AVENUE,...,NA,NA,NA,NA,4456314,Passenger Vehicle,Unknown/Other,NA,NA,NA
5,04/14/2021,12:47,BRONX,10463,-999.000000,-999.0000,NA,MAJOR DEEGAN EXPRESSWAY RAMP,NA,NA,...,Unspecified,NA,NA,NA,4407458,Truck,Passenger Vehicle,NA,NA,NA


In [91]:
# Export cleaned dataframe to a csv file
df_cleaned.to_csv(f'/Users/shashankkashi/Downloads/Motor_Vehicle_Collisions_-_Crashes_Cleaned.csv', index=False)
# Change to your own directory

In [63]:
df_cleaned.shape

(1794430, 29)